In [1]:
import torch
import random
import calendar

import platform
import sys

# Проверка доступности GPU (как мы делали раньше)
# Если CUDA есть, используем её. Если нет - процессор.

print(f"torch.version.cuda: {torch.version.cuda}")       # Должно показать '12.4' или '12.1' (не None!)
print(f"torch.cuda.is_available(): {torch.cuda.is_available()}") # Должно быть True
print(f"torch.xpu.is_available(): {torch.xpu.is_available()}") 

def get_device():
    if torch.cuda.is_available():
        return torch.device("cuda")
    # Проверяем атрибут xpu, чтобы код не падал на машинах со старым PyTorch
    elif hasattr(torch, 'xpu') and torch.xpu.is_available():
        return torch.device("xpu")
    else:
        return torch.device("cpu")

device = get_device()


print(f"✅ Устройство для вычислений: {device}")
# Если XPU активен, полезно вывести имя карты:
if device.type == 'xpu':
    print(f"   Карта: {torch.xpu.get_device_name(0)}")
elif device.type == 'cuda':
    print(f"   Карта: {torch.cuda.get_device_name(0)}")


torch.version.cuda: None
torch.cuda.is_available(): False
torch.xpu.is_available(): True
✅ Устройство для вычислений: xpu
   Карта: Intel(R) Graphics [0x7d45]


In [2]:
print(f"sys.version: {sys.version}")

print(f"platform.python_version(): {platform.python_version()}")

sys.version: 3.12.3 (main, Nov  6 2025, 13:44:16) [GCC 13.3.0]
platform.python_version(): 3.12.3


In [3]:
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug',
          'Sep', 'Oct', 'Nov', 'Dec']


def generate_date():
    year = random.randint(1950, 2050)

    monthIdx = random.randint(1, 12)

    match monthIdx:
        case 1 | 3 | 5 | 7 | 8 | 10 | 12:
            day = random.randint(1, 31)
        case 4 | 6 | 9 | 11:
            day = random.randint(1, 30)
        case 2:
            if calendar.isleap(year):
                day = random.randint(1, 29)
            else:
                day = random.randint(1, 28)

    # 1. Порядок: Год - Месяц - День
    # 2. Форматирование :02d добавляет ноль в начале, если число < 10 (например, 05)
    tgt_str = f'{year}-{monthIdx:02d}-{day:02d}'

    # Вход оставляем как есть: "25 Jan 2023"
    src_str = f'{day} {months[monthIdx-1]} {year}'

    # Добавляем маркеры старта и стопа
    tgt_str = f'<{tgt_str}>'

    return src_str, tgt_str


print(generate_date())

('22 Sep 2005', '<2005-09-22>')


In [4]:
# 0. Определяем специальные токены и их индексы (зафиксируем их для удобства)
# Важно: 0-й индекс часто резервируют для PAD, так как он используется по умолчанию
# при инициализации тензоров нулями.
PAD_token = 0
SOS_token = 1 
EOS_token = 2

# Символы, которые гарантированно будут в нашем словаре
special_chars = {
    '<PAD>': PAD_token, 
    '<SOS>': SOS_token, 
    '<EOS>': EOS_token
}


# 1. Определяем все возможные символы (Alphabet)
# set() убирает дубликаты
chars = set("0123456789 abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ<>-")

# 2. Объединяем символы и специальные токены для создания словаря char2idx
# Начинаем индексацию с 3 (после 0, 1, 2)
sorted_data_chars = sorted(chars)
next_index = len(special_chars)

# 3. Создаем Map: Символ -> Индекс (Integer)
# sorted() сортирует, чтобы порядок всегда был одинаковым
# enumerate() возвращает пары (счетчик, элемент)
char2idx = special_chars.copy()
char2idx.update({char: idx + next_index for idx, char in enumerate(sorted_data_chars)})
print(f"char2idx: {char2idx}")

# 3. Создаем обратный Map: Индекс -> Символ (для расшифровки ответа)
idx2char = {idx: char for char, idx in char2idx.items()}

print(f"idx2char: {idx2char}")


# Добавляем служебный токен "EOS" (End Of Sentence) или PAD, если нужно
# Пока просто запомним размер нашего "алфавита"
vocab_size = len(char2idx)

print(f"Размер словаря (количество уникальных символов): {vocab_size}")
print(f"Индекс буквы 'A': {char2idx['A']}")
print(f"Символ под индексом 10: {idx2char[10]}")
print(f"Размер словаря с токенами: {vocab_size}")
print(f"Индекс <PAD>: {char2idx['<PAD>']}")
print(f"Индекс <EOS>: {char2idx['<EOS>']}")

char2idx: {'<PAD>': 0, '<SOS>': 1, '<EOS>': 2, ' ': 3, '-': 4, '0': 5, '1': 6, '2': 7, '3': 8, '4': 9, '5': 10, '6': 11, '7': 12, '8': 13, '9': 14, '<': 15, '>': 16, 'A': 17, 'B': 18, 'C': 19, 'D': 20, 'E': 21, 'F': 22, 'G': 23, 'H': 24, 'I': 25, 'J': 26, 'K': 27, 'L': 28, 'M': 29, 'N': 30, 'O': 31, 'P': 32, 'Q': 33, 'R': 34, 'S': 35, 'T': 36, 'U': 37, 'V': 38, 'W': 39, 'X': 40, 'Y': 41, 'Z': 42, 'a': 43, 'b': 44, 'c': 45, 'd': 46, 'e': 47, 'f': 48, 'g': 49, 'h': 50, 'i': 51, 'j': 52, 'k': 53, 'l': 54, 'm': 55, 'n': 56, 'o': 57, 'p': 58, 'q': 59, 'r': 60, 's': 61, 't': 62, 'u': 63, 'v': 64, 'w': 65, 'x': 66, 'y': 67, 'z': 68}
idx2char: {0: '<PAD>', 1: '<SOS>', 2: '<EOS>', 3: ' ', 4: '-', 5: '0', 6: '1', 7: '2', 8: '3', 9: '4', 10: '5', 11: '6', 12: '7', 13: '8', 14: '9', 15: '<', 16: '>', 17: 'A', 18: 'B', 19: 'C', 20: 'D', 21: 'E', 22: 'F', 23: 'G', 24: 'H', 25: 'I', 26: 'J', 27: 'K', 28: 'L', 29: 'M', 30: 'N', 31: 'O', 32: 'P', 33: 'Q', 34: 'R', 35: 'S', 36: 'T', 37: 'U', 38: 'V', 39

In [5]:
import torch.nn as nn


class EncoderRNN_Batch(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN_Batch, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        
        # Грузим всю последовательность сразу
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        # input shape: (Seq_Len, Batch_Size)  <-- Было (1, 1)
        
        # 1. Embeddings
        # Раньше мы делали .view(1, 1, -1). Теперь это не нужно.
        # PyTorch сам поймет: (12 строк, 32 примера) -> (12, 32, 128)
        embedded = self.embedding(input)
        
        # 2. GRU
        # Мы скармливаем ЕМУ ВСЮ последовательность сразу!
        # output: (Seq_Len, Batch_Size, Hidden_Size) - состояния на каждом шаге
        # hidden: (1, Batch_Size, Hidden_Size) - последнее состояние (суть всей фразы)
        output, hidden = self.gru(embedded, hidden)
        
        return output, hidden

    def initHidden(self, batch_size):
        # Теперь мы создаем память не для одного примера, а для 32 сразу
        return torch.zeros(1, batch_size, self.hidden_size, device=device)

print("✅ Batch Encoder готов.")

✅ Batch Encoder готов.


In [6]:
import torch.nn.functional as F

class AttnDecoderRNN_Batch(nn.Module):
    def __init__(self, hidden_size, output_size, max_length=15):
        super(AttnDecoderRNN_Batch, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        
        # Слои остались теми же, что и раньше
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        # input: (1, Batch_Size) - текущие буквы для всех примеров в батче
        # hidden: (1, Batch_Size, Hidden) - память с прошлого шага
        # encoder_outputs: (Seq_Len, Batch, Hidden) - ВАЖНО! Тут время первое.
        
        # 1. Embeddings
        # (1, Batch) -> (1, Batch, Hidden)
        embedded = self.embedding(input)

        # 2. Считаем Attention Weights
        # Нам нужно склеить (Batch, Hidden) и (Batch, Hidden).
        # Берем [0], чтобы избавиться от первого измерения (1).
        attn_input = torch.cat((embedded[0], hidden[0]), 1) 
        
        # (Batch, Hidden*2) -> (Batch, Max_Len)
        attn_weights = F.softmax(self.attn(attn_input), dim=1)

        # 3. Применяем Attention (Матричное умножение)
        # BMM требует формат (Batch, X, Y).
        # encoder_outputs у нас (Seq_Len, Batch, Hidden). Надо перевернуть!
        # .transpose(0, 1) делает (Batch, Seq_Len, Hidden)
        encoder_outputs_transposed = encoder_outputs.transpose(0, 1)
        
        # (Batch, 1, Max_Len) x (Batch, Max_Len, Hidden) -> (Batch, 1, Hidden)
        attn_applied = torch.bmm(attn_weights.unsqueeze(1),
                                 encoder_outputs_transposed)

        # 4. Combine & GRU
        # Склеиваем (Batch, Hidden) и (Batch, Hidden) -> (Batch, Hidden*2)
        output = torch.cat((embedded[0], attn_applied[:, 0, :]), 1)
        
        output = self.attn_combine(output).unsqueeze(0) # -> (1, Batch, Hidden)
        output = F.relu(output)
        
        output, hidden = self.gru(output, hidden)

        # 5. Output
        output = F.log_softmax(self.out(output[0]), dim=1)
        
        return output, hidden, attn_weights

print("✅ Batch Decoder готов.")

✅ Batch Decoder готов.


In [7]:
# Маркер конца строки (End Of Sentence).
# В нашем генераторе мы оборачиваем дату в <...>.
# Значит, символом остановки будет '>'.
EOS_token = char2idx['>']

def tensorFromSentence(sentence):
    # 1. Превращаем строку в список индексов
    indexes = [char2idx[char] for char in sentence]
    
    # 2. Добавляем маркер конца (опционально, но полезно для декодера)
    # В нашем случае '>' уже есть в конце строки от генератора, 
    # но для надежности можно убедиться, что он там есть.
    # Наш генератор выдает '<...>', так что '>' уже внутри indexes.
    
    # 3. Превращаем в Тензор
    # dtype=torch.long обязателен для индексов (Embedding слой требует long)
    # .view(-1, 1) делает из вектора столбик (Sequence Length x Batch Size)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

In [8]:
def train_batch(input_batch, target_batch, encoder, decoder, 
                encoder_optimizer, decoder_optimizer, criterion):
    
    # Сброс градиентов
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # Размеры
    batch_size = input_batch.size(1)
    target_len = target_batch.size(0) # Длина самой длинной фразы в ответе

    # 1. Запуск Энкодера
    # Инициализируем память сразу для всей пачки
    encoder_hidden = encoder.initHidden(batch_size)
    
    # Прогоняем весь батч разом
    encoder_outputs, encoder_hidden = encoder(input_batch, encoder_hidden)
    
    # FIX ДЛЯ ATTENTION:
    # Энкодер мог вернуть длину 12, а Декодер ждет 15 (max_length).
    # Создаем тензор-заглушку нужного размера.
    # (Max_Len, Batch, Hidden)
    proj_encoder_outputs = torch.zeros(decoder.max_length, batch_size, encoder.hidden_size, device=device)
    
    # Копируем реальные данные внутрь заглушки
    actual_len = encoder_outputs.size(0)
    proj_encoder_outputs[:actual_len, :, :] = encoder_outputs

    # 2. Запуск Декодера
    # Начальный вход: строка из <SOS> токенов (размер = batch_size)
    decoder_input = torch.tensor([[SOS_token] * batch_size], device=device) # (1, Batch)
    decoder_hidden = encoder_hidden # Наследуем память
    
    loss = torch.tensor(0.0, device=device)

    # Teacher Forcing: применяем сразу ко всему батчу (для простоты и скорости)
    # В промышленных решениях иногда миксуют, но для обучения это ок.
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Кормим правильными ответами
        for di in range(target_len):
            decoder_output, decoder_hidden, _ = decoder(
                decoder_input, decoder_hidden, proj_encoder_outputs)
            
            # target_batch[di] - это срез (Batch_Size) правильных букв для текущего шага
            loss += criterion(decoder_output, target_batch[di])
            
            # Следующий вход - правильный срез (unsqueeze добавляет измерение 1, чтобы стало 1xBatch)
            decoder_input = target_batch[di].unsqueeze(0) 
            
    else:
        # Кормим предсказаниями
        for di in range(target_len):
            decoder_output, decoder_hidden, _ = decoder(
                decoder_input, decoder_hidden, proj_encoder_outputs)
            
            loss += criterion(decoder_output, target_batch[di])
            
            # Выбираем лучшие буквы
            topv, topi = decoder_output.topk(1)
            # topi имеет размер (Batch, 1). Транспонируем в (1, Batch) для следующего шага
            decoder_input = topi.transpose(0, 1).detach()

    # 3. Backpropagation
    loss.backward()

    # Важно: клиппинг градиентов (чтобы не взрывались от больших батчей)
    torch.nn.utils.clip_grad_norm_(encoder.parameters(), 5.0)
    torch.nn.utils.clip_grad_norm_(decoder.parameters(), 5.0)

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_len

In [9]:
# Настройки
TOTAL_SAMPLES = 10000  # Сколько всего примеров хотим
TRAIN_RATIO = 0.8      # 80% на обучение

# 1. Генерируем уникальный пул данных
# Используем set, чтобы исключить дубликаты (одинаковые даты)
dataset_set = set()

print(f"🔄 Генерируем {TOTAL_SAMPLES} уникальных пар...")

while len(dataset_set) < TOTAL_SAMPLES:
    # Вызываем твою функцию (она должна быть уже определена выше)
    pair = generate_date() 
    dataset_set.add(pair)

# Превращаем в список и перемешиваем
full_dataset = list(dataset_set)
random.shuffle(full_dataset)

# 2. Разрезаем на Train и Test
split_index = int(TOTAL_SAMPLES * TRAIN_RATIO)

train_data = full_dataset[:split_index]
test_data = full_dataset[split_index:]

print(f"✅ Готово!")
print(f"📚 Обучающая выборка (Train): {len(train_data)} шт.")
print(f"🎓 Тестовая выборка (Test):  {len(test_data)} шт. (Модель их никогда не увидит)")
print(f"Пример из теста: {test_data[0]}")

🔄 Генерируем 10000 уникальных пар...
✅ Готово!
📚 Обучающая выборка (Train): 8000 шт.
🎓 Тестовая выборка (Test):  2000 шт. (Модель их никогда не увидит)
Пример из теста: ('19 Jul 2045', '<2045-07-19>')


In [10]:
from torch.nn.utils.rnn import pad_sequence

# Вспомогательная функция: Строка -> Список индексов
def indexesFromSentence(sentence):
    # Удаляем старые маркеры < и >, если они есть, чтобы не путаться
    clean_sentence = sentence.replace('<', '').replace('>', '')
    
    # Превращаем в цифры
    # Если вдруг встретим незнакомый символ, пропускаем его (или падаем, но пока пропустим)
    return [char2idx[char] for char in clean_sentence if char in char2idx] + [EOS_token]

def get_batch(batch_size=32):
    input_list = []
    target_list = []
    
    # 1. Набираем batch_size случайных примеров
    for _ in range(batch_size):
        pair = random.choice(train_data) # Берем из обучающей выборки
        input_str, target_str = pair
        
        # Превращаем в списки чисел (НЕ тензоры пока)
        input_idxs = indexesFromSentence(input_str)
        target_idxs = indexesFromSentence(target_str)
        
        # Важно: для PyTorch конвертируем списки в тензоры
        input_list.append(torch.tensor(input_idxs, dtype=torch.long, device=device))
        target_list.append(torch.tensor(target_idxs, dtype=torch.long, device=device))
        
    # 2. Магия PAD_SEQUENCE
    # Она берет список тензоров разной длины и делает из них один прямоугольный тензор.
    # padding_value=PAD_token (наш 0) — чем заполнять пустоту.
    # ВАЖНО: По умолчанию она делает размерность (Max_Length, Batch_Size).
    # Это то, что нужно для RNN (время идет вниз, батчи идут вправо).
    input_batch = pad_sequence(input_list, padding_value=PAD_token)
    target_batch = pad_sequence(target_list, padding_value=PAD_token)
    
    return input_batch, target_batch

# === ТЕСТ ===
# Давай посмотрим, как выглядит один батч
inp, tgt = get_batch(batch_size=3)
print(f"Форма входного батча (Seq_Len, Batch): {inp.shape}")
print(f"Форма целевого батча (Seq_Len, Batch): {tgt.shape}")
print("\nПример входного тензора (видим нули-паддинги в конце коротких фраз?):")
print(inp.transpose(0, 1)) # Транспонируем для удобного просмотра (Batch, Seq)

Форма входного батча (Seq_Len, Batch): torch.Size([12, 3])
Форма целевого батча (Seq_Len, Batch): torch.Size([11, 3])

Пример входного тензора (видим нули-паддинги в конце коротких фраз?):
tensor([[ 8,  5,  3, 31, 45, 62,  3,  7,  5,  7, 14, 16],
        [ 7,  6,  3, 20, 47, 45,  3,  7,  5,  7, 11, 16],
        [ 7, 14,  3, 35, 47, 58,  3,  7,  5,  9, 13, 16]], device='xpu:0')


In [11]:
# 1. Создаем батч из 5 примеров
BATCH_SIZE = 5
inp_batch, tgt_batch = get_batch(BATCH_SIZE)
# inp_batch имеет размер (Seq_Len, 5)

# 2. Создаем энкодер
test_enc_batch = EncoderRNN_Batch(vocab_size, hidden_size=128).to(device)

# 3. Инициализируем память (теперь передаем batch_size!)
hidden = test_enc_batch.initHidden(BATCH_SIZE)

# 4. Прогоняем
enc_output, enc_hidden = test_enc_batch(inp_batch, hidden)

print(f"Входной батч: {inp_batch.shape} (Seq_Len, Batch)")
print(f"Выход всех шагов (Output): {enc_output.shape} (Seq_Len, Batch, Hidden)")
print(f"Финальная память (Hidden): {enc_hidden.shape} (1, Batch, Hidden)")

# Проверка: Batch dimension (вторая цифра) везде должна быть 5
assert enc_output.shape[1] == BATCH_SIZE
assert enc_hidden.shape[1] == BATCH_SIZE
print("✅ Тест пройден: Энкодер съел матрицу и не поперхнулся.")

Входной батч: torch.Size([12, 5]) (Seq_Len, Batch)
Выход всех шагов (Output): torch.Size([12, 5, 128]) (Seq_Len, Batch, Hidden)
Финальная память (Hidden): torch.Size([1, 5, 128]) (1, Batch, Hidden)
✅ Тест пройден: Энкодер съел матрицу и не поперхнулся.


In [12]:
# 1. Параметры
BATCH_SIZE = 5
MAX_LEN = 15 # Должно совпадать с max_length в Декодере (у нас там хардкод 15 пока)

# 2. Создаем модели
test_enc = EncoderRNN_Batch(vocab_size, 128).to(device)
test_dec = AttnDecoderRNN_Batch(128, vocab_size, max_length=MAX_LEN).to(device)

# 3. Готовим данные
# Вход: (12, 5) - длина 12, батч 5
inp, tgt = get_batch(BATCH_SIZE) 

# 4. Прогон Энкодера
enc_out, enc_hidden = test_enc(inp, test_enc.initHidden(BATCH_SIZE))
# enc_out сейчас имеет размер (Реальная_Длина, 5, 128).
# НО! Декодер ждет, что encoder_outputs будет ровно MAX_LENGTH (15).
# В реальном проекте это решается масками, но мы сделаем Pad прямо здесь для теста.
# Создаем тензор нужного размера (15, 5, 128) и копируем туда данные.
pad_enc_out = torch.zeros(MAX_LEN, BATCH_SIZE, 128, device=device)
# Копируем только реальную длину
length = enc_out.size(0)
pad_enc_out[:length, :, :] = enc_out

# 5. Прогон Декодера (Один шаг)
dec_input = torch.tensor([[SOS_token] * BATCH_SIZE], device=device) # (1, 5)
dec_hidden = enc_hidden # Передаем память

dec_out, dec_hidden_new, attn = test_dec(dec_input, dec_hidden, pad_enc_out)

print(f"Decoder Input shape: {dec_input.shape}")
print(f"Decoder Output shape: {dec_out.shape} (должно быть Batch x Vocab)")
print(f"Attention shape: {attn.shape} (должно быть Batch x Max_Len)")

assert dec_out.shape == (BATCH_SIZE, vocab_size)
assert attn.shape == (BATCH_SIZE, MAX_LEN)
print("✅ Тест пройден: Декодер работает с пакетами!")


Decoder Input shape: torch.Size([1, 5])
Decoder Output shape: torch.Size([5, 69]) (должно быть Batch x Vocab)
Attention shape: torch.Size([5, 15]) (должно быть Batch x Max_Len)
✅ Тест пройден: Декодер работает с пакетами!


In [ ]:
import torch.optim as optim
import time

# === НАСТРОЙКИ ===
BATCH_SIZE = 2048  # Теперь мы учим по 64 примера за раз!
n_iters = 2000   # Итераций меньше, т.к. за одну итерацию мы видим 64 примера
learning_rate = 0.005 # Чуть уменьшим, так как градиенты суммируются
teacher_forcing_ratio = 0.5

# 1. Модели
encoder_batch = EncoderRNN_Batch(vocab_size, 128).to(device)
decoder_batch = AttnDecoderRNN_Batch(128, vocab_size, max_length=15).to(device)

encoder_batch.compile()
decoder_batch.compile()

# 2. Оптимизаторы
enc_optim = optim.Adam(encoder_batch.parameters(), lr=learning_rate) # Попробуем Adam, он быстрее сходится
dec_optim = optim.Adam(decoder_batch.parameters(), lr=learning_rate)

# 3. Функция потерь с ИГНОРОМ ПАДДИНГОВ
# Это критически важно! Мы не штрафуем модель за предсказание пустоты на месте пустоты.
criterion = nn.NLLLoss(ignore_index=PAD_token)

plot_losses = []
print_every = 1
current_loss = 0
start = time.time()

print(f"🚀 Запуск BATCH обучения (Batch Size={BATCH_SIZE})...")

for iter in range(1, n_iters + 1):
    # Получаем пачку данных
    input_batch, target_batch = get_batch(BATCH_SIZE)
    
    # Шаг обучения
    loss = train_batch(input_batch, target_batch, encoder_batch, decoder_batch,
                       enc_optim, dec_optim, criterion)
    
    current_loss += loss
    plot_losses.append(loss)

    if iter % print_every == 0:
        print_loss_avg = current_loss / print_every
        current_loss = 0
        # Считаем скорость (примеров в секунду)
        # iter * batch_size = сколько примеров прошли
        speed = (iter * BATCH_SIZE) / (time.time() - start)
        print(f'{iter} | Loss: {print_loss_avg:.4f} | Speed: {speed:.1f} samples/sec')

print("✅ Обучение завершено.")

🚀 Запуск BATCH обучения (Batch Size=2048)...


W1203 14:57:16.438000 30025 torch/_inductor/utils.py:1613] [1/0_1] Not enough SMs to use max_autotune_gemm mode


In [ ]:
import numpy as np

# Функция сглаживания (свертка)
def moving_average(data, window_size=50):
    # window_size=50 означает, что каждая точка графика - это среднее за 50 шагов
    return np.convolve(data, np.ones(window_size)/window_size, mode='valid')

# Применяем фильтр к твоему списку ошибок
smooth_losses = moving_average(all_losses)

# Рисуем
plt.figure(figsize=(10, 5))
plt.plot(smooth_losses, label='Smoothed Loss')
plt.title("График обучения (Скользящее среднее)")
plt.xlabel("Итерации")
plt.ylabel("Ошибка")
plt.grid(True, alpha=0.3) # Добавим сетку для наглядности
plt.legend()
plt.show()

In [ ]:
def evaluate(input_sentence):
    with torch.no_grad(): # Отключаем градиенты для теста
        input_tensor = tensorFromSentence(input_sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        # Массив для хранения весов внимания (для картинки)
        decoder_attentions = torch.zeros(decoder.max_length, decoder.max_length)

        encoder_outputs = torch.zeros(decoder.max_length, encoder.hidden_size, device=device)

        # 1. Читаем (Encoder)
        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] = encoder_output[0, 0]

        # 2. Пишем (Decoder)
        decoder_input = torch.tensor([[char2idx['<']]], device=device)
        decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(decoder.max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            
            # Сохраняем веса внимания
            decoder_attentions[di] = decoder_attention.data
            
            topv, topi = decoder_output.data.topk(1)
            
            if topi.item() == char2idx['>']:
                break # Конец фразы
            else:
                decoded_words.append(idx2char[topi.item()])

            decoder_input = topi.detach()

        return "".join(decoded_words), decoder_attentions[:di + 1]

In [ ]:
def evaluate_randomly(n=5):
    print(f"\n🎓 ЭКЗАМЕН (Данные из Test Set):")
    correct = 0
    
    for i in range(n):
        # Берем из TEST data
        pair = random.choice(test_data)
        src = pair[0]
        tgt_real = pair[1]
        
        # Просим модель предсказать
        output_words, _ = evaluate(src) # Используем нашу функцию evaluate
        
        # Сравниваем (убираем маркеры < > для красоты)
        tgt_clean = tgt_real.replace('<','').replace('>','')
        out_clean = output_words.replace('<','').replace('>','')
        
        # Простая проверка на точное совпадение
        is_match = "✅" if tgt_clean == out_clean else f"❌ (Ожидалось: {tgt_clean})"
        if tgt_clean == out_clean: correct += 1
            
        print(f"Вход: {src.ljust(15)} -> Выход: {out_clean} {is_match}")
        
    print(f"\nТочность на выборке: {correct}/{n}")

# Запускаем экзамен
evaluate_randomly(10)

In [ ]:
import matplotlib.ticker as ticker

def showAttention(input_sentence):
    output_words, attentions = evaluate(input_sentence)
    
    # Рисуем тепловую карту
    fig = plt.figure(figsize=(10,5))
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.cpu().numpy(), cmap='bone')
    fig.colorbar(cax)

    # Подписи осей
    ax.set_xticklabels([''] + list(input_sentence), rotation=90)
    ax.set_yticklabels([''] + list(output_words))
    
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()
    print(f"Вход:  {input_sentence}")
    print(f"Выход: {output_words}")

# === ЗАПУСК ПРОВЕРКИ ===
# Проверяем на датах, которых модель НЕ видела
def showAttentions(n = 3):
    for i in range(n):
        pair = random.choice(test_data)
        src = pair[0]
        showAttention(src)
